# InceptionV3 

In [2]:
import numpy as np
import tensorflow as tf
import keras
from keras.layers import Input, Conv2D, MaxPool2D, Concatenate, AveragePooling2D, Dropout, Dense, Flatten, BatchNormalization
from keras.models import Model

## The Model

In [3]:
def ConvBN(X_input, filters, kernel_r, kernel_c, stride, padding='valid'):
    X = Conv2D(filters=filters, 
               kernel_size=(kernel_r, kernel_c), 
               strides=(stride, stride), 
               padding=padding, 
               activation='relu')(X_input)
    return BatchNormalization()(X)

In [4]:
def Inception1(X_input):
    X1 = ConvBN(X_input, 64, 1, 1, 1)
    X1 = ConvBN(X1, 96, 3, 3, 1, 'same')
    X1 = ConvBN(X1, 96, 3, 3, 1, 'same')
    
    X2 = ConvBN(X_input, 48, 1, 1, 1)
    X2 = ConvBN(X2, 64, 3, 3, 1, 'same')
    
    X3 = AveragePooling2D(pool_size=(3,3), strides=(1,1), padding='same')(X_input)
    X3 = ConvBN(X3, 64, 1, 1, 1)
    
    X4 = ConvBN(X_input, 64, 1, 1, 1)
    
    return Concatenate(axis=-1)([X1, X2, X3, X4])

In [5]:
def Inception2(X_input):
    X1 = ConvBN(X_input, 128, 1, 1, 1)
    X1 = ConvBN(X1, 128, 1, 7, 1, 'same')
    X1 = ConvBN(X1, 128, 7, 1, 1, 'same')
    X1 = ConvBN(X1, 128, 1, 7, 1, 'same')
    X1 = ConvBN(X1, 192, 7, 1, 1, 'same')
    
    X2 = ConvBN(X_input, 128, 1, 1, 1)
    X2 = ConvBN(X2, 128, 1, 7, 1, 'same')
    X2 = ConvBN(X2, 192, 7, 1, 1, 'same')
    
    X3 = AveragePooling2D(pool_size=(3,3), strides=(1,1), padding='same')(X_input)
    X3 = ConvBN(X3, 192, 1, 1, 1)
    
    X4 = ConvBN(X_input, 192, 1, 1, 1)
    
    return Concatenate(axis=-1)([X1, X2, X3, X4])

In [6]:
def Inception3(X_input):
    X1 = ConvBN(X_input, 448, 1, 1, 1)
    X1 = ConvBN(X1, 384, 3, 3, 1, 'same')
    X11 = ConvBN(X1, 384, 1, 3, 1, 'same')
    X12 = ConvBN(X1, 384, 3, 1, 1, 'same')
    
    X2 = ConvBN(X_input, 384, 1, 1, 1)
    X21 = ConvBN(X2, 384, 1, 3, 1, 'same')
    X22 = ConvBN(X2, 384, 3, 1, 1, 'same')
    
    X3 = AveragePooling2D(pool_size=(3,3), strides=(1,1), padding='same')(X_input)
    X3 = ConvBN(X3, 192, 1, 1, 1)
    
    X4 = ConvBN(X_input, 320, 1, 1, 1)
    
    return Concatenate(axis=-1)([X11, X12, X21, X22, X3, X4])

In [7]:
def DimReduction1(X_input):
    X1 = ConvBN(X_input, 64, 1, 1, 1)
    X1 = ConvBN(X1, 96, 3, 3, 1, 'same')
    X1 = ConvBN(X1, 96, 3, 3, 2)
    
    X2 = ConvBN(X_input, 384, 3, 3, 2)
    
    X3 = MaxPool2D(pool_size=(3,3), strides=(2,2))(X_input)
    
    return Concatenate(axis=-1)([X1, X2, X3])

In [8]:
def DimReduction2(X_input):
    X1 = ConvBN(X_input, 192, 1, 1, 1)
    X1 = ConvBN(X1, 192, 1, 7, 1, 'same')
    X1 = ConvBN(X1, 192, 7, 1, 1, 'same')
    X1 = ConvBN(X1, 192, 3, 3, 2)
    
    X2 = ConvBN(X_input, 192, 1, 1, 1)
    X2 = ConvBN(X2, 320, 3, 3, 2)
    
    X3 = MaxPool2D(pool_size=(3,3), strides=(2,2))(X_input)
    
    return Concatenate(axis=-1)([X1, X2, X3])

In [9]:
# Model architecture

X_input = Input(shape=(299,299,3))

X = ConvBN(X_input,32,3,3,2)
X = ConvBN(X,32,3,3,1)
X = ConvBN(X,64,3,3,1,'same')
X = MaxPool2D(pool_size=(3,3), strides=(2,2))(X)

X = ConvBN(X,80,3,3,1)
X = ConvBN(X,192,3,3,2)
X = ConvBN(X,288,3,3,1,'same')

X = Inception1(X)
X = Inception1(X)
X = Inception1(X)

X = DimReduction1(X)

X = Inception2(X)
X = Inception2(X)
X = Inception2(X)
X = Inception2(X)
X = Inception2(X)

X = DimReduction2(X)

X = Inception3(X)
X = Inception3(X)

X = tf.keras.layers.GlobalAveragePooling2D()(X)
X = Dense(units=1000, activation='softmax')(X)

inceptionV3 = Model(inputs=X_input, outputs=X)
inceptionV3.summary()

Model: "functional_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 299, 299, 3) 0                                            
__________________________________________________________________________________________________
conv2d (Conv2D)                 (None, 149, 149, 32) 896         input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization (BatchNorma (None, 149, 149, 32) 128         conv2d[0][0]                     
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, 147, 147, 32) 9248        batch_normalization[0][0]        
_______________________________________________________________________________________

In [10]:
inceptionV3.save("InceptionV3.keras")

# Data Preparation

In [11]:
TRAIN_DIR = "../input/imagenetmini-1000/imagenet-mini/train/"
VAL_DIR = "../input/imagenetmini-1000/imagenet-mini/val/"
batch_size=64

from keras.preprocessing.image import ImageDataGenerator

train_generator = ImageDataGenerator(horizontal_flip=True)
val_generator = ImageDataGenerator(horizontal_flip=True)

train_gen = train_generator.flow_from_directory(TRAIN_DIR, 
                                               target_size=(299,299),
                                               batch_size=batch_size,
                                               class_mode='categorical')

val_gen = val_generator.flow_from_directory(VAL_DIR, 
                                            target_size=(299,299),
                                            batch_size=batch_size,
                                            class_mode='categorical')

Found 34745 images belonging to 1000 classes.
Found 3923 images belonging to 1000 classes.


## Model Training

In [12]:
inceptionV3.compile(loss='categorical_crossentropy', 
                   optimizer='adam',
                   metrics=['accuracy'])

In [13]:
from livelossplot import PlotLossesKeras

inceptionV3.fit_generator(train_gen,
                        epochs=5,
                        steps_per_epoch=34745//batch_size,
                        validation_data=val_gen, 
                        validation_steps=3923//batch_size, 
                        callbacks=[PlotLossesKeras()],
                        verbose=1)

ValueError: could not broadcast input array from shape (299,299,3) into shape (299,299,3,3)